# Project: Build a Document Retriever Search Engine on Wikipedia Data

## Install OpenAI, and LangChain dependencies

In [ ]:
!pip install langchain==0.2.0
!pip install langchain-openai==0.1.7
!pip install langchain-community==0.2.0
!pip install sentence-transformers==2.7.0

## Install Chroma Vector DB and LangChain wrapper

In [ ]:
!pip install langchain-chroma

## Enter Open AI API Key

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Setup Environment Variables

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [ ]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

## Vector Databases

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector database takes care of storing embedded data and performing vector search for you.

### Chroma Vector DB

[Chroma](https://docs.trychroma.com/getting-started) is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

### Get the wikipedia data

In [ ]:
# if you can't download using the following code
# go to https://drive.google.com/file/d/1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW download it
# manually upload it on colab
!gdown 1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW

In [ ]:
import gzip
import json

wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

docs = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())

        #Add all paragraphs
        #passages.extend(data['paragraphs'])

        #Only add the first paragraph
        docs.append({
                        'metadata': {
                                        'title': data.get('title'),
                                        'article_id': data.get('id')
                        },
                        'data': ' '.join(data.get('paragraphs')[0:3]) # restrict data to first 3 paragraphs to run later modules faster
        })

print("Passages:", len(docs))

In [ ]:
# We subset our data so we only use a subset of wikipedia documents to run things faster
docs = [doc for doc in docs for x in ['linguistics', 'india', 'cheetah']
              if x in doc['data'].lower().split()]

In [ ]:
len(docs)

In [ ]:
docs[:3]

### Create LangChain Documents

In [ ]:
from langchain.docstore.document import Document

docs = [Document(page_content=doc['data'],
                 metadata=doc['metadata']) for doc in docs]

In [ ]:
docs[:3]

In [ ]:
len(docs)

### Split larger documents into smaller chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
chunked_docs = splitter.split_documents(docs)

In [ ]:
chunked_docs[:3]

In [ ]:
len(chunked_docs)

### Create a Vector DB and persist on disk

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [ ]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db = Chroma.from_documents(documents=chunked_docs,
                                  collection_name='wikipedia_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./wikipedia_db")

### Load Vector DB from disk

This is just to show once you have a vector database on disk you can just load and create a connection to it anytime

In [ ]:
# load from disk
chroma_db = Chroma(persist_directory="./wikipedia_db",
                   collection_name='wikipedia_db',
                   embedding_function=openai_embed_model)

In [ ]:
chroma_db

## Experiment with Vector Database Retrievers

Here we will explore the following retrieval strategies on our Vector Database:

- Similarity or Ranking based Retrieval
- Multi Query Retrieval
- Contextual Compression Retrieval
- Chained Retrieval Pipeline

### Similarity or Ranking based Retrieval

We use cosine similarity here and retrieve the top 3 similar documents based on the user input query

In [ ]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 3})

In [ ]:
query = "what is the capital of India?"
top3_docs = similarity_retriever.invoke(query)
top3_docs

In [ ]:
query = "what is the old capital of India?"
top3_docs = similarity_retriever.invoke(query)
top3_docs

In [ ]:
query = "what is the fastest animal?"
top3_docs = similarity_retriever.invoke(query)
top3_docs

In [ ]:
query = "what is the slowest animal?"
top3_docs = similarity_retriever.invoke(query)
top3_docs

In [ ]:
query = "what is the study of language?"
top3_docs = similarity_retriever.invoke(query)
top3_docs

### Multi Query Retrieval

Retrieval may produce different results with subtle changes in query wording, or if the embeddings do not capture the semantics of the data well. Prompt engineering / tuning is sometimes done to manually address these problems, but can be tedious.

The [`MultiQueryRetriever`](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents.

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
# Set logging for the queries
import logging

similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 3})

mq_retriever = MultiQueryRetriever.from_llm(
    retriever=similarity_retriever, llm=chatgpt
)

logging.basicConfig()
# so we can see what queries are generated by the LLM
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
query = "what is the capital of India?"
docs = mq_retriever.invoke(query)
docs

In [ ]:
query = "what is the old capital of India?"
docs = mq_retriever.invoke(query)
docs

In [ ]:
query = "what is the study of language?"
docs = mq_retriever.invoke(query)
docs

### Contextual Compression Retrieval

The information most relevant to a query may be buried in a document with a lot of irrelevant text. Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this. The idea is simple: instead of immediately returning retrieved documents as-is, you can compress them using the context of the given query, so that only the relevant information is returned.

This compression can happen in the form of:

- Remove parts of the content of retrieved documents which are not relevant to the query. This is done by extracting only relevant parts of the document to the given query

- Filter out documents which are not relevant to the given query but do not remove content from the document

Here we wrap our multi-query retriever with a `ContextualCompressionRetriever`. Then we'll add an `LLMChainExtractor`, which will iterate over the initially returned documents and extract from each only the content that is relevant to the query.

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


# extracts from each document only the content that is relevant to the query
compressor = LLMChainExtractor.from_llm(llm=chatgpt)

# retrieves the documents similar to query and then applies the compressor
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=mq_retriever
)

In [ ]:
query = "what is the financial capital of India?"
docs = compression_retriever.invoke(query)
docs

In [ ]:
query = "what is the old capital of India?"
docs = compression_retriever.invoke(query)
docs

In [ ]:
query = "what is the fastest animal?"
docs = compression_retriever.invoke(query)
docs

In [ ]:
query = "what is the slowest animal?"
docs = compression_retriever.invoke(query)
docs

In [ ]:
query = "what is the study of language?"
docs = compression_retriever.invoke(query)
docs

The `LLMChainFilter` is slightly simpler but more robust compressor that uses an LLM chain to decide which of the initially retrieved documents to filter out and which ones to return, without manipulating the document contents.

In [ ]:
from langchain.retrievers.document_compressors import LLMChainFilter

#  decides which of the initially retrieved documents to filter out and which ones to return
_filter = LLMChainFilter.from_llm(llm=chatgpt)

# retrieves the documents similar to query and then applies the filter
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=mq_retriever
)

In [ ]:
query = "what is the financial capital of India?"
docs = compression_retriever.invoke(query)
docs

In [ ]:
query = "what is the old capital of India?"
docs = compression_retriever.invoke(query)
docs

In [ ]:
query = "what is the fastest animal?"
docs = compression_retriever.invoke(query)
docs

In [ ]:
query = "what is the slowest animal?"
docs = compression_retriever.invoke(query)
docs

### Chained Retrieval Pipeline

This strategy uses a chain of multiple retrievers sequentially to get to the most relevant documents. The following is the flow

Similarity Retrieval → Compression Filter → Reranker Model Retrieval

In [ ]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

# Retriever 1 - simple cosine distance based retriever
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 5})

#  decides which of the initially retrieved documents to filter out and which ones to return
_filter = LLMChainFilter.from_llm(llm=chatgpt)
# Retriever 2 - retrieves the documents similar to query and then applies the filter
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=similarity_retriever
)

# download an open-source reranker model - BAAI/bge-reranker-v2-m3
reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-large")
reranker_compressor = CrossEncoderReranker(model=reranker, top_n=3)
# Retriever 3 - Uses a Reranker model to rerank retrieval results from the previous retriever
final_retriever = ContextualCompressionRetriever(
    base_compressor=reranker_compressor, base_retriever=compressor_retriever
)

In [ ]:
query = "what is the financial capital of India?"
docs = final_retriever.invoke(query)
docs

In [ ]:
query = "what is the old capital of India?"
docs = final_retriever.invoke(query)
docs

In [ ]:
query = "what is the fastest animal?"
docs = final_retriever.invoke(query)
docs

In [ ]:
query = "what is the slowest animal?"
docs = final_retriever.invoke(query)
docs

In [ ]:
query = "what is the study of language?"
docs = final_retriever.invoke(query)
docs

In [ ]:
query = "what is chess?"
docs = final_retriever.invoke(query)
docs